In [1]:
from selenium import webdriver 
from bs4 import BeautifulSoup 
from selenium.webdriver.common.by import By

# 명시적 대기를 위해 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
import pymysql

conn = pymysql.connect(host='172.16.221.55', user='test', password='1234',
                       db='test', charset='utf8', port=4306)  # DB 연결

curs = conn.cursor()   # SQL 쿼리 실행

In [3]:
# 상품 정보를 담는 클레스
class TourInfo:
    # 맴버변수 (실제 컬럼보다는 작게 세팅했음)
    title = ''
    price = ''
    info  = ''
    link  = ''
    img   = ''
    contents = ''
    tournum = ''
    # 생성자
    def __init__(self, tournum, title, price, info, link, img, contents=None ):
        self.tournum = tournum
        self.title = title
        self.price = price
        self.info  = info
        self.link  = link
        self.img   = img
        self.contents = contents

In [4]:
insert_sql = """
            insert into tourinfo
            (keyword, title, price, info, link, img)
            values(%s, %s, %s, %s, %s, %s)
            """

select_sql = """
            select last_insert_id()
            """

In [5]:
# 상품 정보를 담는 리스트 (TourInfo 리스트)
tour_list = []

In [10]:
main_url = 'http://tour.interpark.com/' 

keyword = input("여행지:")

driver = webdriver.Chrome(executable_path='C:/python_edu/program/chromedriver.exe')

driver.get(main_url)

# 검색창을 찾아서 검색어 입력
# id : SearchGNBText
driver.find_element_by_id('SearchGNBText').send_keys(keyword)

# 검색 버튼 클릭
driver.find_element_by_css_selector('button.search-btn').click()

#5초 대기
time.sleep(5)

# 해외여행 더보기 버튼 클릭
driver.find_element_by_css_selector('.oTravelBox>.boxList>.moreBtnWrap>.moreBtn').click()

#5초 대기
time.sleep(5)

#검색 개수를 출력하는 객체 리턴
totalCount = driver.find_element_by_css_selector('#totalAllCnt')

#텍스트 출력
print("검색 결과:", totalCount.text)

#, 를 삭제하고 정수로 변환
totalCount = int(totalCount.text.replace(",",""))

#전체 페이지를 계산
totalPage =  (totalCount//10)+1

totalPage

여행지:로마
검색 결과: 205


21

In [7]:
comment_insert_sql = """
                    insert into tourcomment (tournum, ctitle, cgrade, ctx)
                    values(%s, %s, %s, %s)
                    """

In [12]:
import traceback

In [14]:
for page in range(1, totalPage+1):
    try:
        # 자바스크립트 구동하기
        driver.execute_script("searchModule.SetCategoryList(%s, '')" % page)
        time.sleep(2)
        print("%s 페이지 이동" % page)
        
        boxItems = driver.find_elements_by_css_selector('.oTravelBox>.boxList>li')
        # 상품 하나 하나 접근
        for li in boxItems:
            img = li.find_element_by_css_selector('img').get_attribute('src') 
            link = li.find_element_by_css_selector('a').get_attribute('onclick')
            title = li.find_element_by_css_selector('h5.proTit').text 
            comment = li.find_element_by_css_selector('.proSub').text  # .클래스속성
            price =   li.find_element_by_css_selector('.proPrice').text 
            info = ''
            for proinfo in li.find_elements_by_css_selector('.info-row .proInfo'):
                info+=proinfo.text+"\n"
            
            price = price.replace(",", "").replace("원~","")
            
            print("title:%s\nlink:%s\nimg:%s\ncomment:%s\nprice:%s\ninfo:%s" % (title, link, img, comment, price, info))
            
            curs.execute(insert_sql,
                         (keyword, title, price, info, link, img.split("/")[-1]))
            conn.commit()
            
            curs.execute(select_sql)
            
            #데이타 Fetch
            rows = curs.fetchall()
            tournum = 0
            for row in rows:
                tournum = row
                
            print("tournum:", tournum)
            
            print('='*100)
            obj = TourInfo(
                tournum,
                title,
                price,
                info,
                link,
                img
            )
            tour_list.append( obj )
    except Exception as e1:
        traceback.print_stack()  # 에러 났을 때 추적해서 보여줌

1 페이지 이동
title:《euro》로마 바티칸 Plus 투어(전일)
link:searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=J3010019','')
img:http://tourimage.interpark.com/product/tour/00161/J30/280/J3010019_1_700.JPG
comment:
price:35000 
info:여행 기간 : 0박1일로마
출발 가능 기간 : 2019.07.16~2019.10.31
평점 8.2
12개의 상품평



  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\tornado\platform\asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\asyncio\base_events.py", line 421, in run_forever
    self._run_once()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\asyncio\ba

2 페이지 이동
title:《euro》로마 바티칸 Plus 투어(전일)
link:searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=J3010019','')
img:http://tourimage.interpark.com/product/tour/00161/J30/280/J3010019_1_700.JPG
comment:
price:35000 
info:여행 기간 : 0박1일로마
출발 가능 기간 : 2019.07.16~2019.10.31
평점 8.2
12개의 상품평



  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\tornado\platform\asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\asyncio\base_events.py", line 421, in run_forever
    self._run_once()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\asyncio\ba

3 페이지 이동
title:《euro》로마 바티칸 Plus 투어(전일)
link:searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=J3010019','')
img:http://tourimage.interpark.com/product/tour/00161/J30/280/J3010019_1_700.JPG
comment:
price:35000 
info:여행 기간 : 0박1일로마
출발 가능 기간 : 2019.07.16~2019.10.31
평점 8.2
12개의 상품평



...

  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\tornado\platform\asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\asyncio\base_events.py", line 421, in run_forever
    self._run_once()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\asyncio\ba

20 페이지 이동
title:《euro》로마 바티칸 Plus 투어(전일)
link:searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=J3010019','')
img:http://tourimage.interpark.com/product/tour/00161/J30/280/J3010019_1_700.JPG
comment:
price:35000 
info:여행 기간 : 0박1일로마
출발 가능 기간 : 2019.07.16~2019.10.31
평점 8.2
12개의 상품평



  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\tornado\platform\asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\asyncio\base_events.py", line 421, in run_forever
    self._run_once()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\asyncio\ba

21 페이지 이동
title:《euro》로마 바티칸 Plus 투어(전일)
link:searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=J3010019','')
img:http://tourimage.interpark.com/product/tour/00161/J30/280/J3010019_1_700.JPG
comment:
price:35000 
info:여행 기간 : 0박1일로마
출발 가능 기간 : 2019.07.16~2019.10.31
평점 8.2
12개의 상품평



  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\site-packages\tornado\platform\asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\asyncio\base_events.py", line 421, in run_forever
    self._run_once()
  File "C:\ProgramData\Anaconda3\envs\python_edu\lib\asyncio\ba

In [50]:
curs = conn.cursor(pymysql.cursors.DictCursor) #default: 인덱스 값으로 조회 결과 리턴
# 이렇게 설정하면 이름으로 조회 결과 리턴

In [57]:
curs.execute(select_sql)

219

In [58]:
rows = curs.fetchall()

In [61]:
for row in rows[0:10]:
    print("row:", row)
    print("tournum:", row['tournum'])
    print("title:", row['title'])
    print("="*100)

row: {'tournum': 6, 'title': '[반반한유럽] 로마 세미팩 7일', 'img': 'B3011862_3_510.jpg', 'info': '여행 기간 : 5박7일로마5\n출발 가능 기간 : 2019.07.16~2019.09.15\n평점 0.0\n0개의 상품평\n', 'link': "searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=B3011862','')", 'keyword': '로마', 'price': 2250000}
tournum: 6
title: [반반한유럽] 로마 세미팩 7일
row: {'tournum': 7, 'title': 'L.o.v.e. I.n 스위스 ♥이탈리아 자유허니문 8일', 'img': 'D3010405_1_680.jpg', 'info': '여행 기간 : 5박8일인천-경유지-인터라켄-베니스-로마-경유지-인천\n출발 가능 기간 : 2019.07.16~2020.03.29\n평점 10.0\n1개의 상품평\n', 'link': "searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=D3010405','')", 'keyword': '로마', 'price': 2691100}
tournum: 7
title: L.o.v.e. I.n 스위스 ♥이탈리아 자유허니문 8일
row: {'tournum': 13, 'title': '[혼자도 괜찮아★4국10일]아시아나항공+로마야간투어포함', 'img': 'A3012938_13_633.jpg', 'info': '여행 기간 : 8박10일영국-프랑스-스위스-이탈리아\n출발 가능 기간 : 2019.07.18~2020.03.17\n평점 9.1\n275개의 상품평\n', 'link': "searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A

In [45]:
tournum = 0

In [101]:
print(len(tour_list) )

22


In [102]:
tour_list[0]

In [17]:
for tour in tour_list[5:10]:
    arr = tour.link.split(',')
    if arr:
        # 대체
        link = arr[0].replace('searchModule.OnClickDetail(','')
        # 슬라이싱 => 앞에 ', 뒤에 ' 제거
        detail_url = link[1:-1]
        # 상세 페이지 이동 : URL 값이 완성된 형태인지 확인 (http~)
        driver.get( detail_url )
        time.sleep(2)
        driver.execute_script("MovingPoint('CM')")
        time.sleep(2)
        #try:
        tits = driver.find_elements_by_css_selector('div.article strong.tit')
        grades = driver.find_elements_by_css_selector('div.article span.grade')
        soup = BeautifulSoup( driver.page_source, 'html.parser')
        txt = soup.select("div.article div.cont")

        ctx=""
        for i in range(0, len(tits)):
            ctitle = tits[i].text
            cgrade = grades[i].text
            ctxts = txt[i].select("p")
            for i in range(0, len(ctxts)):
                 ctx+=str(ctxts[i]).replace("<p>","").replace("</p>","")
            print("title:%s\nrank:%s\nctxts:%s" %(ctitle,cgrade,ctx))

            curs.execute(comment_insert_sql,
                         (tour.tournum, ctitle, cgrade, ctx))
            conn.commit()
            
            
        #except Exception as e1:
            

title:"비바 중남미"
rank:10.00
ctxts: 


InternalError: (1241, 'Operand should contain 1 column(s)')

In [8]:
conn.close()

In [15]:
print(tour.tournum, ctitle, cgrade, ctx)

(334, '페루', '[노팁/노쇼핑] 원더풀 중남미! 3개국 페루+쿠바+멕시코 11일', 4596200, '여행 기간 : 7박11일인천-멕시코시티-리마-쿠스코-우르밤바-마추픽추-아바나-바라데로-멕시코시티-인천\n출발 가능 기간 : 2019.07.21~2019.10.27\n평점 8.9\n24개의 상품평\n', "searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A7010185','')", 'A7010185_2_380.jpg') "비바 중남미" 10.00  


In [ ]:
tour.tournum, ctitle, cgrade, ctx

### 강사님 데이터베이스에 마구 생성

In [38]:
class TourInfo:
# 맴버변수 (실제 컬럼보다는 작게 세팅했음)
    title = ''
    price = ''
    info  = ''
    link  = ''
    img   = ''
    contents = ''
    tournum = ''
# 생성자
def __init__(self, tournum, title, price, info, link, img, contents=None ):
    self.tournum = tournum
    self.title = title
    self.price = price
    self.info  = info
    self.link  = link
    self.img   = img
    self.contents = contents

for keyword in ['모스크바', '예루살렘', '페루', '요르단']:

    main_url = 'http://tour.interpark.com/' 

    driver = webdriver.Chrome(executable_path='C:/python_edu/program/chromedriver.exe')

    driver.get(main_url)

    # 검색창을 찾아서 검색어 입력
    # id : SearchGNBText
    driver.find_element_by_id('SearchGNBText').send_keys(keyword)

    # 검색 버튼 클릭
    driver.find_element_by_css_selector('button.search-btn').click()

    #5초 대기
    time.sleep(5)

    # 해외여행 더보기 버튼 클릭
    driver.find_element_by_css_selector('.oTravelBox>.boxList>.moreBtnWrap>.moreBtn').click()

    #5초 대기
    time.sleep(5)

    #검색 개수를 출력하는 객체 리턴
    totalCount = driver.find_element_by_css_selector('#totalAllCnt')

    #텍스트 출력
    print("검색 결과:", totalCount.text)

    #, 를 삭제하고 정수로 변환
    totalCount = int(totalCount.text.replace(",",""))

    #전체 페이지를 계산
    totalPage =  (totalCount//10)+1

    totalPage
    
    insert_sql = """
            insert into tourinfo
            (keyword, title, price, info, link, img)
            values(%s, %s, %s, %s, %s, %s)
            """

    select_sql = """
                SELECT LAST_INSERT_ID()
                """
    # 상품 정보를 담는 리스트 (TourInfo 리스트)
    tour_list = []

    for page in range(1, totalPage+1):
        try:
            # 자바스크립트 구동하기
            driver.execute_script("searchModule.SetCategoryList(%s, '')" % page)
            time.sleep(2)
            print("%s 페이지 이동" % page)

            boxItems = driver.find_elements_by_css_selector('.oTravelBox>.boxList>li')
            # 상품 하나 하나 접근
            for li in boxItems:
                img = li.find_element_by_css_selector('img').get_attribute('src') 
                link = li.find_element_by_css_selector('a').get_attribute('onclick')
                title = li.find_element_by_css_selector('h5.proTit').text 
                comment = li.find_element_by_css_selector('.proSub').text  # .클래스속성
                price =   li.find_element_by_css_selector('.proPrice').text 
                info = ''
                for proinfo in li.find_elements_by_css_selector('.info-row .proInfo'):
                    info+=proinfo.text+"\n"

                price = price.replace(",", "").replace("원~","")

                print("title:%s\nlink:%s\nimg:%s\ncomment:%s\nprice:%s\ninfo:%s" % (title, link, img, comment, price, info))

                curs.execute(insert_sql,
                             (keyword, title, price, info, link, img.split("/")[-1]))
                conn.commit()

                curs.execute(select_sql)

                #데이타 Fetch
                rows = curs.fetchall()
                tournum = 0
                for row in rows:
                    tournum = row

                print("tournum:", tournum)

                print('='*100)
                obj = TourInfo(
                    tournum,
                    title,
                    price,
                    info,
                    link,
                    img
                )
                tour_list.append( obj )
        except Exception as e1:
            print("error")  # 에러 났을 때 추적해서 보여줌


검색 결과: 23
1 페이지 이동
title:◈2인이상 출발가능◈ [페루일주] 리마/파라카스/이카/나스카/마추픽추/푸노 9일
link:searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=B7010028','')
img:http://tourimage.interpark.com/product/tour/00161/B70/280/B7010028_1_240.jpg
comment:9일간 페루 곳곳을 알차게 둘러보고 오자!!
price:3790000 
info:여행 기간 : 6박9일리마-피스코-이카-나스카-쿠스코-마추픽추-푸노-리마
출발 가능 기간 : 2019.09.01~2020.03.31
평점 0.0
0개의 상품평

tournum: (448,)
error
2 페이지 이동
title:[남미핵심 4개국] NO옵션/NO쇼핑_우유니사막/마추픽추/이과수폭포/리오예수상 14일
link:searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A7010251','')
img:http://tourimage.interpark.com/product/tour/00161/A70/280/A7010251_1_987.jpg
comment:[9/7,27 출발확정]국민/삼성카드 결제시 5% 청구할인
price:7490000 
info:여행 기간 : 11박14일인천-LA-리마-쿠스코-라파스-우유니-라파스-부에노스-이과수-리우-LA-인천
출발 가능 기간 : 2019.07.18~2019.12.27
평점 8.5
4개의 상품평

tournum: (449,)
error
3 페이지 이동
title:『황홀한 경험』노팁/노옵션! 완벽한 남미여행 6개국 18일
link:searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A7010213','')
img:ht